In [1]:
import numpy as np
import os
import zipfile
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import torch

from sklearn import metrics
import cv2
from collections import Counter as cnt
from torchvision import transforms
from keras import Sequential
from torch import nn
from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, confusion_matrix
from torch import optim

from Mymodule.ModelHandler import *
from Mymodule.Utils import *
from Mymodule.GradCam import *
from Mymodule.BatchHandler import *


KeyboardInterrupt



## Visualize Grad-Cam Images

In [ ]:
device = torch.device('cuda:0')
model_name = 'vgg16'

In [ ]:
def explort_imgs(images, parent_dir,candidates):
    for i,image in enumerate(images):
        file_path = os.path.join(parent_dir, f'{candidates[i]}')
        print(file_path)
        cv2.imwrite(file_path, image)
    print('exported..done')
    
def calculate_zero_ratios(img):
    w, h = img.shape
    zeros = 0
    for i in range(w):
        for j in range(w):
            if img[i, j] <= 0:
                zeros += 1
    return zeros / (w*h)

class LayerActivation():
    features = None    
    def __init__(self, model, layer_num):
        self.hook = model.base.features[layer_num].register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output.cpu().data.numpy()
    def remove(self):
        self.hook.remove()

        
activation = {}

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


In [ ]:
root = './Data/Images/'
candidates = os.listdir(root)[1:]
candidates

In [ ]:
images = []
for i, candidate in enumerate(candidates):
    if candidate[-3:] != 'png': continue
        
    image_path = os.path.join(root,candidate)
    image = cv2.imread(image_path)
    print(candidate)
    plt.imshow(image)
    plt.show()
    
    image = cv2.resize(image,(140,140))
    images.append(image)
images = np.array(images).copy()
test_y = np.zeros([16,1])
print(images.shape)
print(test_y.shape)

In [ ]:
test_loader = GetLoader([], images, test_y, batch=len(test_y), test=True)  

In [ ]:
for train_strategy in ['finetuning']:
    for i,(data, label) in enumerate(test_loader):
        data = data
        label = label
        model_save_path = f'./Model/vgg16_{train_strategy}.pt'
        
        model = get_model(model_name, device, pretrained=True)    
        model.load_state_dict(torch.load(model_save_path))
        model.eval()
        
        last_layer = model.base.features[-2]
        cam = GradCAM(model=model, target_layer=last_layer, device=device)
        grayscale_cam = cam(input_tensor=data, target_category=0)
        
    visuals = get_visuals(images/255, grayscale_cam)
    explort_imgs(visuals, f'./Figures/Gradcam/', candidates)

## Visualize model output Feature maps

In [ ]:
class LayerActivation():
    features = None    
    def __init__(self, model, layer_num):
        self.hook = model.base.features[layer_num].register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output.cpu().data.numpy()
    def remove(self):
        self.hook.remove()

        
activation = {}

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

def calculate_zero_ratios(img):
    w, h = img.shape
    zeros = 0
    for i in range(w):
        for j in range(w):
            if img[i, j] <= 0:
                zeros += 1
    return zeros / (w*h)

In [ ]:
root = './Data/Images/'
candidates = os.listdir(root)[1:]
candidates

x_start = 200
y_start = 200
x_end = x_start + 300
y_end = y_start + 400

MRIs = []
images = []
for i, candidate in enumerate(candidates):
    if candidate[-3:] != 'png': continue
    image_path = os.path.join(root,candidate)
        
    image = cv2.imread(image_path)
    image = cv2.resize(image,(140,140))
    images.append(image)
images = np.array(images)
test_y = np.zeros([16,1])
print(images.shape)
print(test_y.shape)

In [ ]:
for i in range(16):
    test_loader = GetLoader([], images[i][None], test_y[i][None], batch=len(test_y), test=True)  
    for train_strategy in ['finetuning']:
        device = torch.device('cuda:1')
        model_name = 'vgg16'
        model = get_model(model_name, device, pretrained=False)
        model_path = f'./Model/vgg16_{train_strategy}.pt'
        model.load_state_dict(torch.load(model_path))
        conv_out = LayerActivation(model, 42)

        for data, label in test_loader:
            data, label = data.to(device), label.to(device)
            labels = label.float()
            logits = model(data)

        conv_out.remove()

        maps = conv_out.features

        fig = plt.figure(figsize=(13,4))
        fig.subplots_adjust(left=0, right=1, bottom=0, top=0.8, hspace=0.1, wspace=0.1)

        mean_zero_ratio = 0.0

        for j in range(512):
            if j < 100:
                ax = fig.add_subplot(5, 20, j+1, xticks=[], yticks=[])
                ax.imshow(maps[0][j])
            mean_zero_ratio += calculate_zero_ratios(maps[0][j])

        mean_zero_ratio /= 512
        plt.savefig(f'./Figures/{candidates[i][:-4]}_heatmap_{mean_zero_ratio}.png')
        plt.show()
        print('%.4f' % mean_zero_ratio)